In [1]:
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np
from numpy.linalg import norm

In [2]:
df = pd.read_csv("fatwa-mui.csv")
df.head()

,No,Judul Fatwa,Tema Fatwa,Nomor Fatwa,Tgl Ditetapkan
0,1,PANDUAN PENYELENGGARAAN IBADAH DI BULAN RAMADA...,Sosial Kemasyarakatan,KEPUTUSAN IJTIMA ULAMA KOMISI FATWA SE-INDONES...,16 December 2003
1,2,PENGGUNAAN MIKROBA DAN PRODUK MIKROBIAL DALAM ...,POM Iptek,01 Tahun 2010,19 January 2010
2,3,AIR DAUR ULANG,POM Iptek,02 Tahun 2010,27 January 2010
3,4,KIBLAT,Ibadah,03 Tahun 2010,1 February 2010
4,5,ARAH KIBLAT,Ibadah,05 Tahun 2010,1 July 2010


In [3]:
import re
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

factory = StemmerFactory()
stemmer = factory.create_stemmer()

def preprocess(text):
    no_punctuation = re.sub(r'[^\w\s]','', text)
    #stem_output = stemmer.stem(no_punctuation)
    return no_punctuation 

In [4]:
df['Sentence'] = df['Judul Fatwa'].apply(preprocess)
print(df[['Judul Fatwa', 'Sentence']].head(10))

                                         Judul Fatwa  \
0  PANDUAN PENYELENGGARAAN IBADAH DI BULAN RAMADA...   
1  PENGGUNAAN MIKROBA DAN PRODUK MIKROBIAL DALAM ...   
2                                     AIR DAUR ULANG   
3                                             KIBLAT   
4                                        ARAH KIBLAT   
5  PENGGUNAAN VAKSIN MENINGITIS BAGI JEMAAH HAJI ...   
6                                         KOPI LUWAK   
7                                         AMIL ZAKAT   
8  PENSUCIAN ALAT PRODUKSI YANG TERKENA NAJIS MUT...   
9  CARA PENSUCIAN EKSTRAK RAGI (YEAST EKSTRACT) D...   

                                            Sentence  
0  PANDUAN PENYELENGGARAAN IBADAH DI BULAN RAMADA...  
1  PENGGUNAAN MIKROBA DAN PRODUK MIKROBIAL DALAM ...  
2                                     AIR DAUR ULANG  
3                                             KIBLAT  
4                                        ARAH KIBLAT  
5  PENGGUNAAN VAKSIN MENINGITIS BAGI JEMAAH HAJI ... 

In [5]:
tokenizer = AutoTokenizer.from_pretrained("indolem/indobert-base-uncased")
model = AutoModel.from_pretrained("indolem/indobert-base-uncased")

In [6]:
def get_embeddings(text, token_length):
    tokens=tokenizer(text,max_length=token_length,padding='max_length',truncation=True)
    output=model(torch.tensor(tokens.input_ids).unsqueeze(0),
                 attention_mask=torch.tensor(tokens.attention_mask).unsqueeze(0))
    return output[1][0].detach().numpy()

In [7]:
def calc_cosine_similarity(array1, array2):
    return np.dot(array1, array2)/(norm(array1)*norm(array2))

In [8]:
n = len(df)
max_length = 40
empty_array = np.zeros((n, n))

In [9]:
vectorize = []
for i in range(n):
    vectorize.append(get_embeddings(df['Judul Fatwa'][i], max_length));

In [10]:
for i in range(len(empty_array[i])):
    for j in range(len(empty_array[i])):
        if (i != j):
          if (empty_array[j][i] == 0):
            similarity = calc_cosine_similarity(vectorize[i],vectorize[j])
            empty_array[i][j] = similarity
          else:
            continue
        else:
          empty_array[i][j] = 1

print(empty_array)

[[1.         0.48819226 0.57455295 ... 0.6199649  0.5013442  0.21256855]
 [0.         1.         0.51580119 ... 0.50838184 0.63107115 0.28830761]
 [0.         0.         1.         ... 0.63391155 0.5606457  0.21805805]
 ...
 [0.         0.         0.         ... 1.         0.63889593 0.32547411]
 [0.         0.         0.         ... 0.         1.         0.27005899]
 [0.         0.         0.         ... 0.         0.         1.        ]]


In [11]:
for i in range(len(empty_array[i])):
    for j in range(len(empty_array[i])):
        if(empty_array[i][j] > 0.7 and empty_array[i][j] < 1):
            empty_array[i][j] = 1
            print(df['Judul Fatwa'][i], ' | ', df['Judul Fatwa'][j], ' | ', empty_array[i][j])
        else:
            empty_array[i][j] = 0

PANDUAN PENYELENGGARAAN IBADAH DI BULAN RAMADAN DAN SYAWAL 1442 H  |  HUKUM MENGHINA DAN MENGKAFIRKAN SAHABAT NABI MUHAMMAD SAW  |  1.0
PANDUAN PENYELENGGARAAN IBADAH DI BULAN RAMADAN DAN SYAWAL 1442 H  |  HUKUM STANDAR SERTIFIKASI HALAL PENYEMBELIHAN HEWAN DENGAN MENGGUNAKAN MESIN  |  1.0
PANDUAN PENYELENGGARAAN IBADAH DI BULAN RAMADAN DAN SYAWAL 1442 H  |  PRINSIP AJARAN ISLAM TENTANG HUBUNGAN ANTAR UMAT BERAGAMA DALAM BINGKAI NEGARA KESATUAN REPUBLIK INDONESIA  |  1.0
PENGGUNAAN MIKROBA DAN PRODUK MIKROBIAL DALAM PRODUK PANGAN  |  STANDAR KEHALALAN PRODUK KOSMETIKA DAN PENGGUNAANNYA  |  1.0
PENGGUNAAN MIKROBA DAN PRODUK MIKROBIAL DALAM PRODUK PANGAN  |  PENGGUNAAN PARTIKEL EMAS DALAM PRODUK KOSMETIKA BAGI LAKI-LAKI  |  1.0
PENGGUNAAN MIKROBA DAN PRODUK MIKROBIAL DALAM PRODUK PANGAN  |  TRANSPLANTASI ORGAN DAN ATAU / JARINGAN TUBUH UNTUK DIRI SENDIRI  |  1.0
PENGGUNAAN MIKROBA DAN PRODUK MIKROBIAL DALAM PRODUK PANGAN  |  PENGGUNAAN HUMAN DIPLOID CELL UNTUK BAHAN PRODUKSI OBAT DAN VAK

In [12]:
labeling = np.genfromtxt("labeling_fatwa2.csv", delimiter=",")
predict = empty_array.flatten()
label = labeling.flatten()

In [13]:
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score
)

precision = precision_score(label, predict)
accuracy = accuracy_score(label, predict)
recall = recall_score(label, predict)
f1score = f1_score(label, predict)

print('Precision ', precision)
print('Accuracy', accuracy)
print('Recal', recall)
print('F1', f1score)

Precision  0.026381909547738693
Accuracy 0.9939196675900277
Recal 0.1693548387096774
F1 0.04565217391304348
